In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

In [ ]:
import numpy as np
from tensorflow.keras.datasets import mnist
from tensorflow.keras.preprocessing.sequence import pad_sequences
from tensorflow.keras.utils import to_categorical
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, LSTM, Embedding, Dropout
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, RepeatVector, TimeDistributed


In [ ]:
# Loading dataset
(X_train, y_train), (X_test, y_test) = mnist.load_data()

In [ ]:
# Preprocessing data
X_train = X_train.reshape(X_train.shape[0], 28, 28, 1)
X_test = X_test.reshape(X_test.shape[0], 28, 28, 1)
X_train = X_train.astype('float32')
X_test = X_test.astype('float32')
X_train /= 255
X_test /= 255

In [ ]:
# CNN model
cnn_model = Sequential([
    
    # Convolutional
    Conv2D(filters=32, kernel_size=(3,3), activation='relu', input_shape=(28,28,1)),
    
    # Max pooling
    MaxPooling2D(pool_size=(2,2)),
    
    # Convolutional
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    
    # Max pooling
    MaxPooling2D(pool_size=(2,2)),
    
    # Convolutional
    Conv2D(filters=64, kernel_size=(3,3), activation='relu'),
    
    # Flattening the output of last convolutional
    Flatten(),
    
    # Dense layer with 64 units
    Dense(units=64, activation='relu')
])

In [ ]:
# LSTM model
lstm_model = Sequential([
    
    # LSTM layer with 128 units
    LSTM(units=128, input_shape=(10,64)),
    
    # Dropout layer with a rate of 0.5
    Dropout(rate=0.5),
    
    # Dense layer with 10 units and softmax activation
    Dense(units=10, activation='softmax')
])

In [ ]:
# Combining CNN and LSTM
model = Sequential([
    cnn_model,
    
    # Repeat the output of CNN model 10
    RepeatVector(n=10),
    
    # Applying a dense layer with 64 units
    TimeDistributed(Dense(units=64)),
    lstm_model
])

In [ ]:
# Compiling the model
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
# Train the model
model.fit(X_train, to_categorical(y_train), epochs=2, batch_size=64)

In [ ]:
# Evaluating the model on test data
loss, accuracy = model.evaluate(X_test, to_categorical(y_test))
print(f'Test accuracy: {accuracy*100:.2f}%')